In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [2]:
import sys
sys.path.insert(0, "../code")   
from utils import generate_cyclical_features, generate_nonoverlap_window_features


In [3]:
%load_ext autoreload
%autoreload 2

In [13]:
dtype_dict={"id":np.uint32,
            "store_nbr":np.uint8,
            "item_nbr":np.uint32,
            "unit_sales":np.float32
           }
df = pd.read_csv("../output/data/train_top_10_store_10_item.csv", dtype=dtype_dict)
cols = ['date', 'store_item', 'store_nbr', 'item_nbr'] + [c for c in df.columns 
                                  if c not in ('date', 'store_item', 'store_nbr', 'item_nbr')]
df = df[cols]
df.rename(columns={"store_nbr": "store", "item_nbr": "item"}, inplace=True)
df["date"] = pd.to_datetime(df["date"])
df.sort_values("date", inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(5)



/var/folders/w3/d06m5l_97yn4bnxd5t6cgbjr0000gn/T/ipykernel_27917/2905051343.py:6: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../output/data/train_top_10_store_10_item.csv", dtype=dtype_dict)


,date,store_item,store,item,unit_sales,onpromotion
0,2013-01-07,48_502331,48,502331,70.00000,NaN
1,2013-01-07,46_584028,46,584028,304.27301,NaN
2,2013-01-07,3_114790,3,114790,28.00000,NaN
3,2013-01-07,3_305229,3,305229,89.00000,NaN
4,2013-01-07,3_314384,3,314384,106.00000,NaN


In [14]:
df.head(5)

,date,store_item,store,item,unit_sales,onpromotion
0,2013-01-07,48_502331,48,502331,70.00000,NaN
1,2013-01-07,46_584028,46,584028,304.27301,NaN
2,2013-01-07,3_114790,3,114790,28.00000,NaN
3,2013-01-07,3_305229,3,305229,89.00000,NaN
4,2013-01-07,3_314384,3,314384,106.00000,NaN


In [15]:
df.dtypes

date           datetime64[ns]
store_item             object
store                   uint8
item                   uint32
unit_sales            float32
onpromotion            object
dtype: object

In [17]:
non_overlap_df =   generate_nonoverlap_window_features(df.query("store_item == '3_114790'"), window_size=15)
non_overlap_df.head()

,store_item,store,item,sales_day_1,store_med_day_1,item_med_day_1,sales_day_2,store_med_day_2,item_med_day_2,sales_day_3,...,item_med_day_12,sales_day_13,store_med_day_13,item_med_day_13,sales_day_14,store_med_day_14,item_med_day_14,sales_day_15,store_med_day_15,item_med_day_15
0,3_114790,3,114790,28.0,28.0,28.0,11.0,11.0,11.0,13.0,...,23.0,27.0,27.0,27.0,35.0,35.0,35.0,21.0,21.0,21.0
1,3_114790,3,114790,14.0,14.0,14.0,17.0,17.0,17.0,14.0,...,33.0,36.0,36.0,36.0,31.0,31.0,31.0,18.0,18.0,18.0
2,3_114790,3,114790,25.0,25.0,25.0,17.0,17.0,17.0,20.0,...,26.0,22.0,22.0,22.0,28.0,28.0,28.0,29.0,29.0,29.0
3,3_114790,3,114790,19.0,19.0,19.0,18.0,18.0,18.0,28.0,...,24.0,18.0,18.0,18.0,24.0,24.0,24.0,17.0,17.0,17.0
4,3_114790,3,114790,19.0,19.0,19.0,38.0,38.0,38.0,35.0,...,18.0,36.0,36.0,36.0,17.0,17.0,17.0,21.0,21.0,21.0


In [16]:
df = generate_cyclical_features(df, window_size=15)

In [9]:
df.query("store_item == '3_114790'").head()

,date,store_item,store,item,unit_sales,onpromotion,day_sin_1,day_cos_1,day_sin_2,day_cos_2,...,day_sin_11,day_cos_11,day_sin_12,day_cos_12,day_sin_13,day_cos_13,day_sin_14,day_cos_14,day_sin_15,day_cos_15
2,2013-01-07,3_114790,3,114790,28.0,0,0.781831,0.623490,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,2013-01-08,3_114790,3,114790,11.0,0,0.974928,-0.222521,0.781831,0.623490,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204,2013-01-09,3_114790,3,114790,13.0,0,0.433884,-0.900969,0.974928,-0.222521,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
306,2013-01-10,3_114790,3,114790,16.0,0,-0.433884,-0.900969,0.433884,-0.900969,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
403,2013-01-11,3_114790,3,114790,15.0,0,-0.974928,-0.222521,-0.433884,-0.900969,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df.to_excel("../output/data//train_top_10_store_10_item_sine_cosine.xlsx",
            index=False)


In [7]:
df = generate_nonoverlap_window_features(df, window_size=15)

In [8]:
df.head(4)

,id,sales_day_1,store_med_day_1,item_med_day_1,yearly_sin_1,yearly_cos_1,quarterly_sin_1,quarterly_cos_1,seasonal_sin_1,seasonal_cos_1,...,yearly_sin_15,yearly_cos_15,quarterly_sin_15,quarterly_cos_15,seasonal_sin_15,seasonal_cos_15,monthly_sin_15,monthly_cos_15,weekly_sin_15,weekly_cos_15
0,3_114790_2013-01-07,28.0,93.947998,25.0,0.103102,0.994671,0.0,1.0,-1.0,-1.836970e-16,...,0.337523,0.941317,0.0,1.0,-1.0,-1.836970e-16,0.0,1.0,0.0,1.0
1,3_305229_2013-01-07,89.0,93.947998,36.0,0.103102,0.994671,0.0,1.0,-1.0,-1.836970e-16,...,0.337523,0.941317,0.0,1.0,-1.0,-1.836970e-16,0.0,1.0,0.0,1.0
2,3_314384_2013-01-07,106.0,93.947998,86.5,0.103102,0.994671,0.0,1.0,-1.0,-1.836970e-16,...,0.337523,0.941317,0.0,1.0,-1.0,-1.836970e-16,0.0,1.0,0.0,1.0
3,3_364606_2013-01-07,125.0,93.947998,118.0,0.103102,0.994671,0.0,1.0,-1.0,-1.836970e-16,...,0.337523,0.941317,0.0,1.0,-1.0,-1.836970e-16,0.0,1.0,0.0,1.0


In [9]:
df.columns

Index(['id', 'sales_day_1', 'store_med_day_1', 'item_med_day_1',
       'yearly_sin_1', 'yearly_cos_1', 'quarterly_sin_1', 'quarterly_cos_1',
       'seasonal_sin_1', 'seasonal_cos_1',
       ...
       'yearly_sin_15', 'yearly_cos_15', 'quarterly_sin_15',
       'quarterly_cos_15', 'seasonal_sin_15', 'seasonal_cos_15',
       'monthly_sin_15', 'monthly_cos_15', 'weekly_sin_15', 'weekly_cos_15'],
      dtype='object', length=196)

In [10]:
df.drop(columns=['id'], inplace=True)

In [12]:
df.to_excel(
    "../output/data/2025_05_22_train_nonoverlap_top_10_store_item.xlsx",
    index=False
)

In [ ]:
# --- Apply MinMax Scaling ---
scaler = MinMaxScaler()
scaled_values = scaler.fit_transform(non_overlap_df.drop(columns=["date","store_item", "store", "item"]))
scaled_df = pd.DataFrame(scaled_values, columns=non_overlap_df.columns[4:])
scaled_df.insert(0, "date", non_overlap_df["date"])
scaled_df.insert(1, "store_item", non_overlap_df["store_item"])
scaled_df.insert(2, "store", non_overlap_df["store"])
scaled_df.insert(3, "item", non_overlap_df["item"])
scaled_df.head()